In [1]:
import numpy as np
import cv2
from time import sleep

In [87]:
def extract_contours(path):
	img = cv2.imread(path)
	maskim = cv2.inRange(img, (0, 0, 200), (255, 255, 255))
	img[maskim != 0] = [0, 255, 0]
	lines = cv2.bitwise_not(img, mask=cv2.inRange(img, (200, 0, 0), (255, 255, 255)))
	blurrim = cv2.GaussianBlur(lines, (13, 13), 0)
	can_edgesim = cv2.Canny(blurrim, 100, 200)

	contoursim, hierarchy = cv2.findContours(can_edgesim, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
	contoursim = np.array(contoursim)
	contoursim = contoursim[np.array([cv2.arcLength(c, False) for c in contoursim]) > 200]

	epsilonsim = [0.1 * cv2.arcLength(contour, True) for contour in contoursim]
	approxsim = [cv2.approxPolyDP(contour, epsilon, True) for contour, epsilon in zip(contours, epsilonsim)]
	return approxsim

recta = extract_contours('recta.png')
curvad = extract_contours('curvad.png')
curvai = extract_contours('curvai.png')
bifurcacion = extract_contours('bifurcacion.png')
interseccion = extract_contours('interseccion.png')
intersecciont = extract_contours('intersecciont.png')
incorporacion = extract_contours('incorporacion.png')

nombre_escenas = ['recta', 'curvad', 'curvai', 'bifurcacion', 'intersecciont', 'incorporacion', 'interseccion']
escenas = [recta, curvad, curvai, bifurcacion, intersecciont, incorporacion, interseccion]

/Users/manuel/.pyenv/versions/3.7.12/lib/python3.7/site-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # Remove the CWD from sys.path while we load stuff.


In [50]:
def hamming_distance(s1, s2):
	res = 0
	for i in range(min(len(s1), len(s2))):
		s1_bits = np.unpackbits(np.array(s1[i], dtype=np.uint8))
		s2_bits = np.unpackbits(np.array(s2[i], dtype=np.uint8))
		res += np.bitwise_xor(s1_bits, s2_bits).sum()
	return res

In [146]:
vid = cv2.VideoCapture('../segmentacion/video4.avi')
while True:
	frame = vid.read()[1]

	mask = cv2.inRange(frame, (0, 0, 200), (255, 255, 255))
	frame[mask != 0] = [0, 255, 0]

	frame[:, 0:4, :] = [255, 0, 0]
	frame[:, -4:, :] = [255, 0, 0]
	frame[0:4, :, :] = [255, 0, 0]
	frame[-4:, :, :] = [255, 0, 0]

	# cv2.imshow('Video', frame)
	blurr = cv2.GaussianBlur(frame, (25, 25), 0)
	can_edges = cv2.Canny(blurr, 100, 200)
	# can_edges[:,0] = 0
	# can_edges[:,-1] = 0
	# can_edges[0,:] = 0
	# can_edges[-1,:] = 0

	contours, hier = cv2.findContours(can_edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	contours = np.array(contours)
	contours = contours[np.array([cv2.arcLength(c, False) for c in contours]) > 200]
	contours = contours[np.array([cv2.contourArea(c) for c in contours]) > 0]
	cv2.drawContours(frame, contours, -1, (255, 255, 255), 4)

	if  len(contours) == 2:
		M = cv2.moments(contours[0])
		cX0 = int(M["m10"] / (M["m00"] +1))
		cY0 = int(M["m01"] / (M["m00"] +1))

		M = cv2.moments(contours[1])
		cX1 = int(M["m10"] / (M["m00"] +1))
		cY1 = int(M["m01"] / (M["m00"] +1))
		order = cY0 < cY1
		bigger = cv2.contourArea(contours[0]) > cv2.contourArea(contours[1]) + 600
		smaler = cv2.contourArea(contours[0]) < cv2.contourArea(contours[1]) - 600

		if bigger and order or smaler and not order:
			cv2.putText(frame, 'curvad', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
		if smaler and order or bigger and not order:
			cv2.putText(frame, 'curvai', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
		else:
			cv2.putText(frame, 'recta', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
	elif len(contours) == 3:
		contours = sorted(contours, key=lambda c: cv2.contourArea(c))

		M = cv2.moments(contours[0])
		cX0 = int(M["m10"] / (M["m00"] +1))
		cY0 = int(M["m01"] / (M["m00"] +1))

		M = cv2.moments(contours[1])
		cX1 = int(M["m10"] / (M["m00"] +1))
		cY1 = int(M["m01"] / (M["m00"] +1))

		M = cv2.moments(contours[2])
		cX2 = int(M["m10"] / (M["m00"] +1))
		cY2 = int(M["m01"] / (M["m00"] +1))

		if cY2 > cY1 and cY2 > cY0:
			cv2.putText(frame, 'interseccionT', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
		else:
			cv2.putText(frame, 'bifurcacion', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
	elif len(contours) == 4:
		cv2.putText(frame, 'intersecciont', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)

	cv2.imshow('Video', frame)
	# cv2.imshow('Canny', can_edges)
	key = cv2.waitKey(1) & 0xFF
	if key == ord('q'):
		break
	if key == ord('s'):
		sleep(1)

vid.release()
cv2.destroyAllWindows()

/Users/manuel/.pyenv/versions/3.7.12/lib/python3.7/site-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [112]:
contours, hier = cv2.findContours(can_edges.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
contours = np.array(contours)
contours = contours[np.array([cv2.arcLength(c, False) for c in contours]) > 350]

epsilons = np.array([0.01 * cv2.arcLength(count, True) for count in contours])
aproximations = np.array([cv2.approxPolyDP(contours[i], epsilons[i], True) for i in range(len(contours))])

cv2.drawContours(frame, contours, -1, (0, 0, 0), 2)

cv2.imshow('Img', frame)
cv2.waitKey()
cv2.destroyAllWindows()

/Users/manuel/.pyenv/versions/3.7.12/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


KeyboardInterrupt: 

In [8]:
blurr = cv2.GaussianBlur(frame, (19, 19), 0)
can_edges = cv2.Canny(blurr, 100, 200)

cv2.imshow('Canny', can_edges)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
orb = cv2.ORB_create()
kp = cv2.KeyPoint()